In [7]:
from torch.utils.data import DataLoader, Dataset
import torch.autograd as autograd
import pickle
import torch.nn.functional as F
import torch.nn as nn

In [8]:
class CharVectorizer():
    def __init__(self, maxlen=10, alphabet="""abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]\{\}"""):
        
        self.alphabet = alphabet
        self.maxlen = maxlen
        self.char_dict = {k: i for i, k in enumerate(self.alphabet, 1)} # indice zero is reserved to blank and unknown characters

    def transform(self,sentences):
        """
        sentences: list of string
        list of review, review is a list of sequences, sequences is a list of int
        """
        sequences = []
        for sentence in sentences:
            seq = [self.char_dict.get(char, 0) for char in sentence[:self.maxlen].lower()]
            sequences.append(seq)                
        return sequences
    
    def get_params(self):
        params = vars(self)
        return params

In [19]:
alphabet="""abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]\{\}"""

int2char = dict(enumerate(alphabet))
char2int = {ch: ii for ii, ch in int2char.items()}

text = 'hellotest'
# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [ ]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot


In [20]:
encoded

array([ 7,  4, 11, 11, 14, 19,  4, 18, 19])

In [ ]:
X = torch.zeros(len(alphabet), self.l0)

In [16]:
# encode the text
encoded = np.array([char2int[ch] for ch in text])

KeyError: 'dasdad'

In [9]:
class CNN(nn.Module):
    def __init__(self, n_classes=2, input_length=1014, input_dim=68,
                 n_conv_filters=256,
                 n_fc_neurons=1024):
        super(CNN, self).__init__()

        self.layer1 = nn.Sequential(nn.Conv1d(input_dim, n_conv_filters, kernel_size=7, padding=0), nn.ReLU(), nn.MaxPool1d(3))
        self.layer2 = nn.Sequential(nn.Conv1d(n_conv_filters, n_conv_filters, kernel_size=7, padding=0), nn.ReLU(), nn.MaxPool1d(3))
        self.layer3 = nn.Sequential(nn.Conv1d(n_conv_filters, n_conv_filters, kernel_size=3, padding=0), nn.ReLU())
        self.layer4 = nn.Sequential(nn.Conv1d(n_conv_filters, n_conv_filters, kernel_size=3, padding=0), nn.ReLU())
        self.layer5 = nn.Sequential(nn.Conv1d(n_conv_filters, n_conv_filters, kernel_size=3, padding=0), nn.ReLU())
        self.layer6 = nn.Sequential(nn.Conv1d(n_conv_filters, n_conv_filters, kernel_size=3, padding=0), nn.ReLU(), nn.MaxPool1d(3))

        # layer 6 output length = (input_length - 96) / 27
        last_cnn_layer_len = (input_length - 96) / 27
        dim = int(last_cnn_layer_len * n_conv_filters)
        self.layer7 = nn.Sequential(nn.Linear(dim, n_fc_neurons), nn.Dropout(0.5))
        self.layer8 = nn.Sequential(nn.Linear(n_fc_neurons, n_fc_neurons), nn.Dropout(0.5))
        self.layer9 = nn.Linear(n_fc_neurons, n_classes)

        if n_conv_filters == 256 and n_fc_neurons == 1024:
            self.__init_weights(mean=0.0, std=0.05)
        elif n_conv_filters == 1024 and n_fc_neurons == 2048:
            self.__init_weights(mean=0.0, std=0.02)

    def __init_weights(self, mean=0.0, std=0.05):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                m.weight.data.normal_(mean, std)
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(mean, std)

    def forward(self, x):

        x = x.transpose(1, 2)

        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)

        out = out.view(out.size(0), -1)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        return out

In [ ]:
class YelpReviews(Dataset):
    def __init__(self, dataset_path, key, l0 = 1014):
        self.dataset_path = dataset_path
        self.key = key # Train, test, val

        # read alphabet
        self.loadAlphabet(alphabet_path)
        self.load(label_data_path)
        
    def __getitem__(self, idx):
        X = self.oneHotEncode(idx)
        y = self.y[idx]
        return X, y

    def load(self, dataset_path, key):
        